# LangChain 0.3 中的 LCEL 详细介绍


LCEL（LangChain Expression Language）是 LangChain 0.3 中的声明式编程框架，用于构建复杂的 AI 应用链。它基于 `Runnable` 接口，提供了强大的组合能力和优化的执行性能。

LCEL 核心概念

LCEL 的核心是 `Runnable` 接口，所有组件都实现了这个接口，支持：
- **同步/异步执行**：`invoke()` 和 `ainvoke()`
- **批量处理**：`batch()` 和 `abatch()`
- **流式处理**：`stream()` 和 `astream()`
- **并行执行**：`RunnableParallel`
- **条件分支**：`RunnableBranch`

In [1]:

## 完整代码示例

"""
LangChain 0.3 LCEL 完整示例集合
基于 LangChain 0.3.26 版本
"""

import asyncio
import json
from typing import Dict, List, Any, Optional
from datetime import datetime

# LangChain 核心组件
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableLambda,
    RunnableParallel,
    RunnableBranch,
    RunnableMap,
    RunnableSequence
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_ollama import OllamaLLM, ChatOllama

# 配置
OLLAMA_BASE_URL = "http://localhost:11434"
OLLAMA_MODEL = "qwen2.5:3b"

def create_llm():
    """创建LLM实例"""
    return OllamaLLM(
        base_url=OLLAMA_BASE_URL,
        model=OLLAMA_MODEL,
        temperature=0.7
    )

def create_chat_llm():
    """创建Chat LLM实例"""
    return ChatOllama(
        base_url=OLLAMA_BASE_URL,
        model=OLLAMA_MODEL,
        temperature=0.7
    )

## 1. 基础 LCEL 链组合

In [2]:

# 1. 基础 LCEL 链组合
def basic_chain_example():
    """基础链组合示例"""
    print("=" * 60)
    print("1. 基础 LCEL 链组合")
    print("=" * 60)

    llm = create_llm()

    # 创建提示模板
    prompt = PromptTemplate.from_template("请用中文回答：{question}")

    # 创建输出解析器
    output_parser = StrOutputParser()

    # 使用 pipe 方法创建链
    chain = prompt | llm | output_parser

    # 调用链
    result = chain.invoke({"question": "什么是人工智能？"})
    print(f"问题：什么是人工智能？")
    print(f"回答：{result}")

    return chain
basic_chain_example()


1. 基础 LCEL 链组合
问题：什么是人工智能？
回答：人工智能（Artificial Intelligence，简称AI）是指由人设计出的一套系统或算法，这套系统或算法能够模拟、扩展和增强人的智能。它让机器能够在没有人类明确编程的情况下学习、推理、解决问题以及完成各种任务。

简单来说，人工智能是一种技术，通过计算机程序和算法来使机器具备理解环境并执行特定任务的能力。这些任务可以包括语音识别、图像处理、语言翻译、玩游戏、自动驾驶汽车等，甚至还能进行创造性活动如艺术创作或音乐生成。

人工智能的核心在于让机器能够模仿人类的智能行为，包括学习能力、推理能力、问题解决能力和创造性的表达方式。它的发展目标是让计算机系统具备类人的认知和决策能力，以达到甚至超越人类的能力。


PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='请用中文回答：{question}')
| OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434')
| StrOutputParser()

## 2. RunnablePassthrough 使用

In [3]:

def passthrough_example():
    """RunnablePassthrough 示例"""
    print("\n" + "=" * 60)
    print("2. RunnablePassthrough 使用")
    print("=" * 60)

    llm = create_llm()

    # 基础透传
    def format_docs(docs):
        return "\n\n".join([f"文档{i+1}: {doc}" for i, doc in enumerate(docs)])

    # 创建链，保留原始输入并添加格式化文档
    chain = (
        RunnablePassthrough.assign(
            formatted_docs=lambda x: format_docs(x["documents"])
        )
        | RunnablePassthrough.assign(
            prompt=lambda x: f"基于以下文档回答问题：\n{x['formatted_docs']}\n\n问题：{x['question']}"
        )
        | (lambda x: x["prompt"])
        | llm
        | StrOutputParser()
    )

    # 测试数据
    input_data = {
        "question": "什么是机器学习？",
        "documents": [
            "机器学习是人工智能的一个分支",
            "它通过算法让计算机从数据中学习",
            "常见的机器学习方法包括监督学习和无监督学习"
        ]
    }

    result = chain.invoke(input_data)
    print(f"问题：{input_data['question']}")
    print(f"回答：{result}")

    return chain
passthrough_example()


2. RunnablePassthrough 使用
问题：什么是机器学习？
回答：机器学习是人工智能的一个分支，它通过算法让计算机从数据中学习。常见的机器学习方法包括监督学习和无监督学习。


RunnableAssign(mapper={
  formatted_docs: RunnableLambda(lambda x: format_docs(x['documents']))
})
| RunnableAssign(mapper={
    prompt: RunnableLambda(lambda x: f"基于以下文档回答问题：\n{x['formatted_docs']}\n\n问题：{x['question']}")
  })
| RunnableLambda(...)
| OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434')
| StrOutputParser()

## 3. RunnableParallel 并行处理

In [4]:
def parallel_example():
    """RunnableParallel 并行处理示例"""
    print("\n" + "=" * 60)
    print("3. RunnableParallel 并行处理")
    print("=" * 60)

    llm = create_llm()

    # 创建不同的分析提示
    sentiment_prompt = PromptTemplate.from_template("分析以下文本的情感（积极/消极/中性）：{text}")
    topic_prompt = PromptTemplate.from_template("提取以下文本的主要话题：{text}")
    summary_prompt = PromptTemplate.from_template("用一句话总结以下文本：{text}")

    # 创建并行分析链
    parallel_chain = RunnableParallel({
        "sentiment": sentiment_prompt | llm | StrOutputParser(),
        "topic": topic_prompt | llm | StrOutputParser(),
        "summary": summary_prompt | llm | StrOutputParser(),
        "original": RunnablePassthrough()
    })

    # 测试文本
    text = "今天天气真好，我和朋友们去公园玩了一整天，感觉非常开心和放松。"

    result = parallel_chain.invoke({"text": text})

    print(f"原文：{text}")
    print(f"情感分析：{result['sentiment']}")
    print(f"主题提取：{result['topic']}")
    print(f"文本摘要：{result['summary']}")

    return parallel_chain
parallel_example()


3. RunnableParallel 并行处理
原文：今天天气真好，我和朋友们去公园玩了一整天，感觉非常开心和放松。
情感分析：这个文本表达的是积极的情感。作者描述了美好的天气以及与朋友一起度过的一整天愉快时光，并且提到了自己感到很开心和放松的状态，这些都表明了积极的情绪体验。
主题提取：这个文本的主要话题是关于作者和他的朋友们在公园度过的一天的愉快经历，重点在于天气良好以及他们感到高兴和放松的状态。主要话题可以总结为“户外休闲与愉悦”。
文本摘要：今天在公园度过的一天因为好天气而格外愉快和放松。


{
  sentiment: PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='分析以下文本的情感（积极/消极/中性）：{text}')
             | OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434')
             | StrOutputParser(),
  topic: PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='提取以下文本的主要话题：{text}')
         | OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434')
         | StrOutputParser(),
  summary: PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='用一句话总结以下文本：{text}')
           | OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434')
           | StrOutputParser(),
  original: RunnablePassthrough()
}

## 4. RunnableBranch 条件分支

In [5]:
def branch_example():
    """RunnableBranch 条件分支示例"""
    print("\n" + "=" * 60)
    print("4. RunnableBranch 条件分支")
    print("=" * 60)

    llm = create_llm()

    # 定义条件函数
    def is_question(x):
        return x["text"].strip().endswith("？") or x["text"].strip().endswith("?")

    def is_greeting(x):
        greetings = ["你好", "hello", "hi", "早上好", "晚上好"]
        return any(greeting in x["text"].lower() for greeting in greetings)

    # 创建不同的处理链
    question_chain = PromptTemplate.from_template("请详细回答这个问题：{text}") | llm
    greeting_chain = PromptTemplate.from_template("友好地回应这个问候：{text}") | llm
    default_chain = PromptTemplate.from_template("请对以下内容进行评论：{text}") | llm

    # 创建分支链
    branch_chain = RunnableBranch(
        (is_question, question_chain),
        (is_greeting, greeting_chain),
        default_chain
    ) | StrOutputParser()

    # 测试不同类型的输入
    test_inputs = [
        {"text": "你好！"},
        {"text": "什么是深度学习？"},
        {"text": "今天天气不错"}
    ]

    for i, input_data in enumerate(test_inputs, 1):
        result = branch_chain.invoke(input_data)
        print(f"输入{i}：{input_data['text']}")
        print(f"输出{i}：{result}\n")

    return branch_chain
branch_example()



4. RunnableBranch 条件分支
输入1：你好！
输出1：你好！很高兴能帮助你。有什么我可以为你效劳的吗？

输入2：什么是深度学习？
输出2：深度学习是一种机器学习的子集，它允许计算机通过构建和训练多层神经网络来自动从大量数据中提取抽象特征。这些层次化的结构模仿了人脑处理信息的方式，即每个“层”可以识别越来越高级别的模式或特征。

在深度学习模型中，输入的数据首先被传递到最底层的神经元，这些神经元会对数据进行初步的非线性变换，从而发现更复杂的、与目标相关的信息。然后，这个结果被传递给下一个层次的神经元，直到达到顶部，即输出层。这种结构被称为多层感知器（Multilayer Perceptron, MLP）。

深度学习在训练过程中使用大量的标注数据来优化其模型参数，以提高预测准确率和泛化能力。常用的深度学习模型包括卷积神经网络（Convolutional Neural Networks, CNNs）、循环神经网络（Recurrent Neural Networks, RNNs）和生成对抗网络（Generative Adversarial Networks, GANs），其中最出名的是前两者。

深度学习已经成为许多领域的关键技术，如自然语言处理、图像识别、语音识别以及预测分析等。通过使用大量的计算资源和高效的数据预处理技术来加速模型训练过程，深度学习已经在这些领域取得了显著的成果，并且正在持续发展之中。

输入3：今天天气不错
输出3：今天的天气确实不错，这通常意味着是一个适宜外出、享受户外活动的好日子。不过具体的感受还会受到其他因素的影响，比如温度、湿度和个人喜好等。总的来说，这是一个令人愉悦的天气条件。如果您有特定地点或具体的要求，请告诉我！



RunnableBranch(branches=[(RunnableLambda(is_question), PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='请详细回答这个问题：{text}')
| OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434')), (RunnableLambda(is_greeting), PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='友好地回应这个问候：{text}')
| OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434'))], default=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='请对以下内容进行评论：{text}')
| OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434'))
| StrOutputParser()

## 5. RunnableLambda 自定义函数

In [ ]:
def lambda_example():
    """RunnableLambda 自定义函数示例"""
    print("\n" + "=" * 60)
    print("5. RunnableLambda 自定义函数")
    print("=" * 60)

    llm = create_llm()

    # 自定义处理函数
    def preprocess_text(x):
        """文本预处理"""
        text = x["text"]
        # 清理文本
        text = text.strip()
        # 添加元数据
        return {
            "processed_text": text,
            "word_count": len(text.split()),
            "char_count": len(text),
            "timestamp": datetime.now().isoformat()
        }

    def postprocess_result(x):
        """结果后处理"""
        return {
            "response": x,
            "processed_at": datetime.now().isoformat(),
            "response_length": len(x)
        }

    # 创建包含自定义函数的链
    chain = (
        RunnableLambda(preprocess_text)
        | RunnablePassthrough.assign(
            ai_response=lambda x: (
                PromptTemplate.from_template("请分析以下文本（{word_count}词，{char_count}字符）：{processed_text}")
                | llm
                | StrOutputParser()
            ).invoke(x)
        )
        | RunnableLambda(lambda x: postprocess_result(x["ai_response"]))
    )

    # 测试
    input_text = {"text": "人工智能正在改变我们的生活方式，从智能手机到自动驾驶汽车。"}
    result = chain.invoke(input_text)

    print(f"输入：{input_text['text']}")
    print(f"AI回应：{result['response']}")
    print(f"处理时间：{result['processed_at']}")
    print(f"回应长度：{result['response_length']}字符")

    return chain
lambda_example()

## 6. 带记忆的对话链

In [ ]:
def memory_chat_example():
    """带记忆的对话链示例"""
    print("\n" + "=" * 60)
    print("6. 带记忆的对话链")
    print("=" * 60)

    chat_llm = create_chat_llm()

    # 创建聊天提示模板
    prompt = ChatPromptTemplate.from_messages([
        ("system", "你是一个友好的AI助手，能够记住之前的对话内容。"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])

    # 创建基础链
    chain = prompt | chat_llm | StrOutputParser()

    # 创建记忆存储
    memory = InMemoryChatMessageHistory()

    # 创建带记忆的链
    chain_with_memory = RunnableWithMessageHistory(
        chain,
        lambda session_id: memory,
        input_messages_key="input",
        history_messages_key="history"
    )

    # 模拟对话
    conversations = [
        "我叫张三，今年25岁",
        "我的爱好是什么？",
        "我多大了？",
        "能总结一下我们的对话吗？"
    ]

    config = {"configurable": {"session_id": "user_123"}}

    for i, message in enumerate(conversations, 1):
        response = chain_with_memory.invoke(
            {"input": message},
            config=config
        )
        print(f"用户{i}：{message}")
        print(f"AI{i}：{response}\n")

    return chain_with_memory

## 7. 复杂数据处理链

In [ ]:
def complex_data_processing():
    """复杂数据处理链示例"""
    print("\n" + "=" * 60)
    print("7. 复杂数据处理链")
    print("=" * 60)

    llm = create_llm()

    # 模拟数据
    data = {
        "users": [
            {"name": "张三", "age": 25, "city": "北京", "interests": ["编程", "阅读"]},
            {"name": "李四", "age": 30, "city": "上海", "interests": ["音乐", "旅行"]},
            {"name": "王五", "age": 28, "city": "深圳", "interests": ["运动", "摄影"]}
        ],
        "query": "分析用户群体特征"
    }

    # 数据处理函数
    def analyze_users(x):
        users = x["users"]
        total_users = len(users)
        avg_age = sum(user["age"] for user in users) / total_users
        cities = list(set(user["city"] for user in users))
        all_interests = []
        for user in users:
            all_interests.extend(user["interests"])

        return {
            "total_users": total_users,
            "average_age": round(avg_age, 1),
            "cities": cities,
            "common_interests": list(set(all_interests)),
            "original_query": x["query"]
        }

    def format_analysis(x):
        return f"""
用户群体分析报告：
- 总用户数：{x['total_users']}人
- 平均年龄：{x['average_age']}岁
- 分布城市：{', '.join(x['cities'])}
- 兴趣爱好：{', '.join(x['common_interests'])}

请基于以上数据回答：{x['original_query']}
"""

    # 创建处理链
    processing_chain = (
        RunnableLambda(analyze_users)
        | RunnableLambda(format_analysis)
        | llm
        | StrOutputParser()
    )

    result = processing_chain.invoke(data)
    print("数据分析结果：")
    print(result)

    return processing_chain

## 8. 异步处理示例

In [ ]:
async def async_example():
    """异步处理示例"""
    print("\n" + "=" * 60)
    print("8. 异步处理示例")
    print("=" * 60)

    llm = create_llm()

    # 创建异步处理链
    prompt = PromptTemplate.from_template("请用中文简要回答：{question}")
    chain = prompt | llm | StrOutputParser()

    # 准备多个问题
    questions = [
        {"question": "什么是机器学习？"},
        {"question": "什么是深度学习？"},
        {"question": "什么是自然语言处理？"},
        {"question": "什么是计算机视觉？"}
    ]

    print("开始异步处理多个问题...")
    start_time = datetime.now()

    # 异步批量处理
    results = await chain.abatch(questions)

    end_time = datetime.now()
    processing_time = (end_time - start_time).total_seconds()

    print(f"处理完成，耗时：{processing_time:.2f}秒\n")

    for i, (question, result) in enumerate(zip(questions, results), 1):
        print(f"问题{i}：{question['question']}")
        print(f"回答{i}：{result}\n")

    return chain

## 9. 流式处理示例

In [ ]:
def streaming_example():
    """流式处理示例"""
    print("\n" + "=" * 60)
    print("9. 流式处理示例")
    print("=" * 60)

    llm = create_llm()

    prompt = PromptTemplate.from_template("请详细解释：{topic}")
    chain = prompt | llm | StrOutputParser()

    print("开始流式生成回答...")
    print("问题：什么是人工智能？")
    print("回答：", end="", flush=True)

    # 流式处理
    for chunk in chain.stream({"topic": "什么是人工智能"}):
        print(chunk, end="", flush=True)

    print("\n\n流式处理完成！")

    return chain

## 10. JSON 输出解析示例

In [ ]:
def json_output_example():
    """JSON输出解析示例"""
    print("\n" + "=" * 60)
    print("10. JSON输出解析示例")
    print("=" * 60)

    llm = create_llm()

    # 创建JSON输出解析器
    json_parser = JsonOutputParser()

    # 创建提示模板，要求JSON格式输出
    prompt = PromptTemplate.from_template("""
请分析以下文本并以JSON格式返回结果：
文本：{text}

请返回包含以下字段的JSON：
- sentiment: 情感（positive/negative/neutral）
- topics: 主要话题列表
- summary: 一句话总结
- word_count: 词数

JSON格式：
""")

    # 创建链
    chain = prompt | llm | json_parser

    # 测试文本
    test_text = "今天的会议非常成功，我们讨论了新产品的开发计划和市场策略，团队成员都很积极参与。"

    try:
        result = chain.invoke({"text": test_text})
        print(f"输入文本：{test_text}")
        print("JSON分析结果：")
        print(json.dumps(result, ensure_ascii=False, indent=2))
    except Exception as e:
        print(f"JSON解析失败：{e}")
        # 降级处理
        simple_chain = prompt | llm | StrOutputParser()
        result = simple_chain.invoke({"text": test_text})
        print(f"文本结果：{result}")

    return chain

In [ ]:

# ============================================================================
# 主函数
# ============================================================================

def main():
    """运行所有示例"""
    print("LangChain 0.3 LCEL 完整示例集合")
    print("基于 LangChain 0.3.26 版本")
    print("确保 Ollama 服务正在运行：http://localhost:11434")

    try:
        # 运行所有同步示例
        basic_chain_example()
        passthrough_example()
        parallel_example()
        branch_example()
        lambda_example()
        memory_chat_example()
        complex_data_processing()
        streaming_example()
        json_output_example()

        # 运行异步示例
        print("\n开始运行异步示例...")
        asyncio.run(async_example())

        print("\n" + "=" * 60)
        print("所有示例运行完成！")
        print("=" * 60)

    except Exception as e:
        print(f"运行出错：{e}")
        print("请确保：")
        print("1. Ollama 服务正在运行")
        print("2. qwen2.5:3b 模型已下载")
        print("3. 网络连接正常")

if __name__ == "__main__":
    main()


# 总结

LCEL 是 LangChain 0.3 中的核心特性，提供了：

1. **声明式编程**：简洁的链式语法
2. **强大组合能力**：灵活的组件组合
3. **性能优化**：自动并行化和批处理
4. **流式支持**：实时响应能力
5. **调试友好**：清晰的执行流程

**选择建议**：
- 简单应用：使用基础链组合
- 复杂逻辑：使用 `RunnableBranch` 和 `RunnableParallel`
- 高性能需求：利用批处理和并行特性
- 复杂状态管理：考虑升级到 LangGraph

所有示例代码都基于 LangChain 0.3.26 版本，确保与您的环境兼容。
